In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [3]:
#from snowflake.ml.modeling.xgboost import XGBRegressor
##from snowflake.ml.modeling.preprocessing import KBinsDiscretizer, OneHotEncoder
#from snowflake.ml.modeling.impute import SimpleImputer

#from snowflake.ml.modeling.compose import ColumnTransformer
#from snowflake.ml.modeling.pipeline import Pipeline
#from snowflake.ml.modeling.preprocessing import StandardScaler, OrdinalEncoder
#from snowflake.ml.modeling.impute import SimpleImputer
#from snowflake.ml.modeling.model_selection import GridSearchCV
#from snowflake.ml.modeling.xgboost import XGBRegressor

import snowflake.snowpark.functions as F

In [4]:
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from time import time
import random
import pandas as pd


In [5]:
#!pip install xgboost==2.0.3
!pip install -q wandb


In [6]:
import wandb
wandb.login(key='fb6bad081ea657a8a2002960c672b0cba554ad15')

run = wandb.init(project ='sf_xgboost_CPU_HM_L')

from wandb.integration.xgboost import WandbCallback

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/rajishah/.netrc
wandb: Currently logged in as: rshah4. Use `wandb login --relogin` to force relogin


wandb: Network error resolved after 2:04:33.364385, resuming normal operation.
wandb: Network error resolved after 2:34:48.294217, resuming normal operation.


In [7]:
snowdf = session.table("tpcds_xgboost.demo.feature_store")
snowdf = snowdf.drop(['CA_ZIP','CUSTOMER_SK', 'C_CURRENT_HDEMO_SK', 'C_CURRENT_ADDR_SK', 'C_CUSTOMER_ID', 'CA_ADDRESS_SK', 'CD_DEMO_SK'])
snowdf.show()

------------------------------------------------------------------------------------------------------------------------------------
|"TOTAL_SALES"  |"C_BIRTH_YEAR"  |"CD_GENDER"  |"CD_MARITAL_STATUS"  |"CD_CREDIT_RATING"  |"CD_EDUCATION_STATUS"  |"CD_DEP_COUNT"  |
------------------------------------------------------------------------------------------------------------------------------------
|30368.88       |1972            |M            |U                    |Good                |4 yr Degree            |0               |
|39077.29       |1976            |M            |U                    |Good                |4 yr Degree            |0               |
|33980.60       |1976            |M            |U                    |Good                |4 yr Degree            |0               |
|40226.64       |1949            |M            |U                    |Good                |4 yr Degree            |0               |
|25446.41       |1952            |M            |U                    

In [8]:
## Dropping any null values
from snowflake.snowpark.functions import col, is_null

# Create a filter condition for non-finite values across all columns
non_finite_filter = None

# Iterate over all columns and update the filter condition
for column in snowdf.columns:
    current_filter = is_null(col(column))
    non_finite_filter = current_filter if non_finite_filter is None else (non_finite_filter | current_filter)

# Apply the filter to the DataFrame to exclude rows with any non-finite values
df_filtered = snowdf.filter(~non_finite_filter)


## Clean up cats
def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))
categorical_cols = ['CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS']
for col in categorical_cols:
    df_filtered = df_filtered.with_column(col, fix_values(col))
    

In [9]:
feature_cols = df_filtered.columns
feature_cols.remove('TOTAL_SALES')
target_col = 'TOTAL_SALES'

snowdf_train, snowdf_test = df_filtered.random_split([0.8, 0.2], seed=82) 
snowdf_train=snowdf_train.limit(1_000)
snowdf_train.count()

snowdf_train_pd  = pd.DataFrame(snowdf_train.collect())
snowdf_train_pd

,TOTAL_SALES,C_BIRTH_YEAR,CD_DEP_COUNT,CD_GENDER,CD_MARITAL_STATUS,CD_CREDIT_RATING,CD_EDUCATION_STATUS
0,31326.00,1992,0,M,D,GOOD,SECONDARY
1,32890.60,1948,0,M,D,GOOD,SECONDARY
2,32934.98,1946,0,M,D,GOOD,SECONDARY
3,31615.52,1929,0,M,D,GOOD,SECONDARY
4,32183.75,1941,0,M,D,GOOD,SECONDARY
...,...,...,...,...,...,...,...
995,32106.74,1969,0,M,D,GOOD,SECONDARY
996,29164.42,1947,0,M,D,GOOD,SECONDARY
997,34850.65,1972,0,M,D,GOOD,SECONDARY
998,27729.11,1970,0,M,D,GOOD,SECONDARY


In [10]:
 ## Distributed Preprocessing - 25X to 50X faster

numeric_features = ['C_BIRTH_YEAR', 'CD_DEP_COUNT']
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

categorical_cols = ['CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_cols)
        ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', XGBRegressor(random_state=123))])

In [11]:
 ## Distributed HyperParameter Optimization
hyper_param = dict(
        model__max_depth=[2,4],
        model__learning_rate=[0.1,0.3],
    )

xg_model = GridSearchCV(
    estimator=pipeline,
    param_grid=hyper_param,
    #cv=5,
    #input_cols=numeric_features + categorical_cols,
    #label_cols=['TOTAL_SALES'],
    #output_cols=["TOTAL_SALES_PREDICT"],
    #verbose=4  ##verbose not working
)

X_train = snowdf_train_pd.drop(columns=['TOTAL_SALES'])
y_train = snowdf_train_pd['TOTAL_SALES']

xg_model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median'))]),
                                                                         ['C_BIRTH_YEAR',
                                                                          'CD_DEP_COUNT']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['CD_GENDER',
                                                                          'CD_MARITAL_STATUS',
                                                                          'CD_CREDIT_RATING',
                                                                          'CD_...
                                                     learning_rate=None,
                                                     max_bin=None,
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=123, ...))]),
             param_grid={'model__learning_rate': [0.1, 0.3],
                         'model__max_depth': [2, 4]})

In [12]:
session.sql('ALTER SESSION SET USE_CACHED_RESULT=FALSE').collect()

[Row(status='Statement executed successfully.')]

In [13]:
lengths = [1_000_000,5_000_000,10_000_000,25_000_000,50_000_000]
random.seed(9001)

for i, length in enumerate(lengths):
    seedv = random.randint(1, 1000)
    snowdf_train, snowdf_test = df_filtered.random_split([0.8, 0.2], seed=seedv)  #82
    snowdf_train=snowdf_train.limit(length)
    snowdf_train_pd  = pd.DataFrame(snowdf_train.collect())
    print (snowdf_train.count())
    X_train = snowdf_train_pd.drop(columns=['TOTAL_SALES'])
    y_train = snowdf_train_pd['TOTAL_SALES']
    init = time()
    xg_model.fit(X_train, y_train)
    total_time = (time() - init) / 60
    print(f'total rows: {length} total time: {total_time} seed: {seedv}')

1000000
total rows: 1000000 total time: 2.7242841323216758 seed: 38
5000000
total rows: 5000000 total time: 13.862894546985626 seed: 90
10000000
total rows: 10000000 total time: 10.111544013023376 seed: 339
25000000
total rows: 25000000 total time: 50.65401263634364 seed: 919
49268458
total rows: 50000000 total time: 169.8913545290629 seed: 399
